In [5]:
import matplotlib.pyplot as plt
import numpy as np
import struct
import os 
import pandas 
import cv2

### Function converting MUMAX3 output files to more usefull form ###

def readOVF(OVF_file):
    f=open(OVF_file,"rb")
    byte=f.read()
    ii=0
    t4=b''
    t8=b''
    while (t8!=b'@\xdew\x83!\x12\xdcB') & (t4!=b'8\xb4\x96I'):
        t8=byte[ii:(ii+8)]
        t4=byte[ii:(ii+4)]
        ii=ii+1
    words=byte[0:ii].decode().replace('\n',' ').split(' ')
    xns="xnodes:"
    yns="ynodes:"
    zns="znodes:"
    bins="Binary"
    xnn=[s+1 for s,v in enumerate(words) if v==xns]
    ynn=[s+1 for s,v in enumerate(words) if v==yns]
    znn=[s+1 for s,v in enumerate(words) if v==zns]
    binn=[s+1 for s,v in enumerate(words) if v==bins]
    xn=int(words[xnn[0]])
    yn=int(words[ynn[0]])
    zn=int(words[znn[0]])
    bn=int(words[binn[0]])
    if bn==4:
        dat=struct.unpack('f'*xn*yn*zn*3,byte[(ii+bn-1):(ii+bn-1+xn*yn*zn*bn*3)])
    if bn==8:
        dat=struct.unpack('d'*xn*yn*zn*3,byte[(ii+bn-1):(ii+bn-1+xn*yn*zn*bn*3)])   
    dat3=np.array(dat).reshape(zn,yn,xn,3)
    ms=np.sqrt(pow(dat3[0,0,0,0],2)+pow(dat3[0,0,0,1],2)+pow(dat3[0,0,0,2],2))
    return dat3/ms

### Setup of output summary table ###

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)

df = pandas.DataFrame(columns=['Q','D','B_ext','number of areas','min','max','average'])   # <-- Creating table template which we will be appending later    

### Defining a loop for reading MUMAX3 output files ###

for litx in range(11):                   
    for lity in range(9):
        for litz in range(41):

            litxx=format(litx*0.2,".1f")            
            litzz=format(litz*0.05,".2f")
            filename=f"Q{litxx}0_D0{lity}mJ_B_ext{litzz}T.ovf" 
            filename=str(filename)
            
            logic=os.path.isfile(filename)

### Defining a loop for finding and numbering non-zero regions in the area ###  

            if logic:
                tab=readOVF(filename) 
                tab2=np.round((tab[0,:,:,2]/2+1/2))
                im = tab2[:]-2
                h,w = im.shape
                mask = np.zeros((h+2,w+2),np.uint8)

                k=0
                for y in range(h):
                    for x in range(w):
                        if im[y,x]==-2:
                            seed=(x,y)
                            num,im,mask,rect = cv2.floodFill(np.float32(im), mask, seed,k, 0,0)
                            k+=1
                            
                print(litxx ,lity, litzz , k)
                
                np.save(filename+".npy", im)
                
### Getting some statistical info from the file ###
                
                 if k>0:
                    vtab=np.zeros(int(np.max(im)+1),dtype=int)
                    xtab=np.zeros(int(np.max(im)+1),dtype=float)
                    ytab=np.zeros(int(np.max(im)+1),dtype=float)
                    r2tab=np.ones(int(np.max(im)+1),dtype=float)

                    for y in range(h):
                        for x in range(w):
                            t=int(im[y,x])
                            if t>=0:
                                vtab[t]+=1
                                xtab[t]=(x+xtab[t]*(vtab[t]-1))/vtab[t]
                                ytab[t]=(y+ytab[t]*(vtab[t]-1))/vtab[t]
                                r2=(x-xtab[t]+0.5)**2+(y-ytab[t]+0.5)**2
                                if r2>r2tab[t]:
                                    r2tab[t]=r2

                    if len(vtab)>0:  
                        minw=np.min(vtab)
                        maxw=np.max(vtab)
                        swaz=np.average(vtab)
                    else:
                        minw=0
                        maxw=0
                        swaz=0

                    df = df.append({'Q':litxx, 'D':lity, 'B_ext':litzz, 'Liczba obszarow':k, 'min':minw, 'max':maxw, 'srednia':swaz}, ignore_index=True)

                    plt.figure(figsize=(20,20))
                    plot1 = plt.subplot2grid((3,3), (0, 0)) 
                    plot2 = plt.subplot2grid((3,3), (0, 1)) 
                    plot3 = plt.subplot2grid((3,3), (1, 0)) 
                    plot4 = plt.subplot2grid((3,3), (1, 1)) 
                    plot4.hist(vtab/r2tab,bins=50,edgecolor="white");
                    plot4.set_title('Histogram r2') 
                    plot3.plot(r2tab,vtab,"o")
                    plot3.plot([0,np.max(r2tab)],[0,max(r2tab)*3.14])
                    plot3.plot([0,np.max(r2tab)],[0,max(r2tab)*2.3])
                    plot3.set_xscale("log")
                    plot3.set_yscale("log")
                    plot3.set_title('Rozklad')
                    plot2.hist(vtab,bins=50,range=(20,500),edgecolor="white");
                    plot2.set_yscale('log')
                    plot2.set_title('Histogram') 
                    plot1.imshow(tab2) 
                    plot1.set_title('Probka')   
                    plt.tight_layout()
                    plt.savefig(filename+ '.png', dpi=300, transparent=False, bbox_inches='tight', facecolor="white")
                    plt.close()

    print(df,file=open("Statistic3.txt","a"))
    df.to_csv('out3.csv', index=True, compression=None)

0.0 6 0.00 203
0.0 6 0.05 205
0.0 6 0.10 205
0.0 6 0.15 205
0.0 6 0.20 205
0.0 6 0.25 206
0.0 6 0.30 205
0.0 6 0.35 205
0.0 6 0.40 201
0.0 6 0.45 200
0.0 6 0.50 198
0.0 6 0.55 197
0.0 6 0.60 197
0.0 6 0.65 195
0.0 6 0.70 195
0.0 6 0.75 197
0.0 6 0.80 202
0.0 6 0.85 279
0.0 6 0.90 433
0.0 6 0.95 600
0.0 6 1.00 294
0.0 6 1.05 185
0.0 6 1.10 185
0.0 6 1.15 185
0.0 6 1.20 186
0.0 6 1.25 186
0.0 6 1.30 185
0.0 6 1.35 185
0.0 6 1.40 184
0.0 6 1.45 184
0.0 6 1.50 183
0.0 6 1.55 182
0.0 6 1.60 182
0.0 6 1.65 180
0.0 6 1.70 180
0.0 6 1.75 181
0.0 6 1.80 181
0.0 6 1.85 181
0.0 6 1.90 181
0.0 6 1.95 181
0.0 6 2.00 180
0.0 7 0.00 236
0.0 7 0.05 236
0.0 7 0.10 236
0.0 7 0.15 235
0.0 7 0.20 235
0.0 7 0.25 235
0.0 7 0.30 235
0.0 7 0.35 235
0.0 7 0.40 235
0.0 7 0.45 235
0.0 7 0.50 235
0.0 7 0.55 234
0.0 7 0.60 234
0.0 7 0.65 234
0.0 7 0.70 231
0.0 7 0.75 230
0.0 7 0.80 231
0.0 7 0.85 232
0.0 7 0.90 230
0.0 7 0.95 230
0.0 7 1.00 231
0.0 7 1.05 228
0.0 7 1.10 232
0.0 7 1.15 295
0.0 7 1.20 330
0.0 7 1.25